In [7]:
Задание 1
Используем файл keywords.csv.
Необходимо написать гео-классификатор, который каждой строке сможет выставить 
географическую принадлежность определенному региону.
Т. е. если поисковый запрос содержит название города региона, 
то в столбце 'region' пишется название этого региона. 
Если поисковый запрос не содержит названия города, то ставим 'undefined'.
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}
Результат классификации запишите в отдельный столбец region.

SyntaxError: invalid syntax (<ipython-input-7-18fab43f9aca>, line 1)

In [4]:
import pandas as pd

In [43]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [44]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [45]:
d = {}

for region in geo_data:
    
    spisok_gorodov = geo_data[region]
    
    d.update({key:value for key,value in zip(spisok_gorodov,
                                [region for i in spisok_gorodov])}) 
 


In [46]:
keywords['region'] = keywords['keyword'].map(d)

In [47]:
keywords['region'] = keywords['region'].fillna('undefined')
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [ ]:
**Задание 2**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class

In [76]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
Total = ratings.merge(movies, on=["movieId"], how='inner')
pvt = Total.pivot_table(index=['title','movieId'], values = 'rating', aggfunc='mean')
pvt.head()

,,rating
title,movieId,
'71 (2014),117867,4.0
'Hellboy': The Seeds of Creation (2004),97757,4.0
'Round Midnight (1986),26564,3.5
'Salem's Lot (2004),27751,5.0
'Til There Was You (1997),779,4.0


In [77]:
pvt=pvt.reset_index()

In [72]:
def classification(i):
    if i  <= 2:
        return 'низкий рейтинг'
    elif i  <= 4 and i >2:
        return 'средний рейтинг'
    elif i  >4.5:
        return 'высокий рейтинг'
    

In [80]:
pvt['class'] = pvt['rating'].apply(classification)
pvt.head()

,title,movieId,rating,class
0,'71 (2014),117867,4.0,средний рейтинг
1,'Hellboy': The Seeds of Creation (2004),97757,4.0,средний рейтинг
2,'Round Midnight (1986),26564,3.5,средний рейтинг
3,'Salem's Lot (2004),27751,5.0,высокий рейтинг
4,'Til There Was You (1997),779,4.0,средний рейтинг


In [ ]:
**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
Ответ дайте в днях.

In [102]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [235]:
ratings_new = ratings.groupby('userId').agg(['min', 'max','count'])['timestamp'].head()
ratings_new['Lifetime'] = ratings_new['max'] - ratings_new['min']
ratings_filt = ratings_new[(ratings_new['count']>100)]
ratings_filt.head()

,min,max,count,Lifetime
userId,,,,
1,964980499,965719662,232,739163
4,945078428,1007574542,216,62496114


In [ ]:
**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". 
Ваша задача проверить это утверждение, 
используя файлы с рейтингами фильмов из материалов занятия. 
Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг 
становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, 
а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга



In [5]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
Total = ratings.merge(movies, on=["movieId"], how='inner')
Total.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [50]:
years = []
for year in range(1950,2010):
    years.append(year)

In [54]:
num = len(years)
j = []
for i in range(1,num+1):
    j.append(i)
print (j)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


In [75]:
def production_year(title):
    for year in years:
        if str(year) in title:
            return year

    return 1900

In [76]:
Total['year'] = Total['title'].apply(production_year)
Total.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [74]:
pvt = Total.pivot_table(index=['year'], values = 'rating', aggfunc='mean')
pvt_sorted = pvt.sort_values(by = ['rating'], ascending = False)
pvt_sorted.head()

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
